## Monolingual Norec experiments with bert
These were originally in the Experiments2 notebook. I have not run them since I moved them here, so maybe there is an undefined variable

In [1]:
from simpletransformers.ner import NERModel, NERArgs
import torch
import pandas as pd
import pickle as pk
import os
import random
import json
import time, datetime
random.seed(64)
# import nltk
import re
# from nltk.tokenize.simple import SpaceTokenizer
from helpers import *
from keep_records import Keep_records

print(f"Cuda: {torch.cuda.is_available()}")

Cuda: True


In [2]:
datafolder = "norne/data/"
train_all = datafolder + "no_bokmaal-ud-train.bmes"
dev_all = datafolder + 'no_bokmaal-ud-dev.bmes'
train_4c = datafolder + "no_bokmaal-ud-train_4c.bmes"
dev_4c = datafolder + 'no_bokmaal-ud-dev_4c.bmes'

dataset_4c = dataset_w_tags(train_4c, dev_4c)
dataset_all = dataset_w_tags(train_all, dev_all)
datasets = [dataset_4c]
keeper = Keep_records("norne")
for ds in datasets:
    print(ds[-1])

['I-PER', 'I-ORG', 'B-PER', 'I-LOC', 'O', 'B-LOC', 'B-ORG', 'I-MISC', 'B-MISC']


In [10]:
# Run bert multilingual with the settings from previous cell


results = []
models = [("bert", 'bert-base-multilingual-cased'), ("xlmroberta", 'xlm-roberta-base'), ("bert", "KB/bert-base-swedish-cased")]
# models = [("bert", "KB/bert-base-swedish-cased")]
# models = [("bert", "NbAiLab/nb-bert-base")] #Fungerte ikke
train_path, dev_path, tags = dataset_4c
for family, transformersmodel in models:
    model_args = NERArgs() # New args loading fall 2020
    model_args.train_batch_size = 16
    
    model_args.num_train_epochs = 8
    model_args.weight_decay = 0.001
    model_args.overwrite_output_dir = True
    model_args.silent = True
    model_args.save_steps = -1
    model_args.evaluate_during_training = False

    model = NERModel(family, transformersmodel, labels = tags, args=model_args)

    out_d = "outputs/"+transformersmodel+"_"+keeper.ts
    model.train_model(train_path, output_dir= out_d, eval_data=dev_path)
    print(transformersmodel, "Done training")

    result, model_outputs, wrong_preds = model.eval_model(dev_path, output_dir=out_d+"/evaluation", silent = True)

    #Record settings and results
    result["train"] = train_path
    result["dev_test"] = dev_path
    result["training_epochs"] = model_args.num_train_epochs
    result["transformer_model"] = transformersmodel

    keeper.keep(result)
    with open ("outputs/"+transformersmodel+"_"+keeper.ts+".pk", "wb") as wf:
        pk.dump(model_outputs, wf)

df = keeper.get_df()

keeper.to_csv(df.round(4))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

bert-base-multilingual-cased Done training
[['O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-st

xlm-roberta-base Done training
[['O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], [

Some weights of the model checkpoint at KB/bert-base-swedish-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at KB/bert-base-swedish-cased and

KB/bert-base-swedish-cased Done training
[['O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O

In [ ]:
predictions, raw_outputs = model.predict(["Sample sentence 1", "Sample sentence 2"])
predictions

[[{'Sample': 'O'}, {'sentence': 'O'}, {'1': 'O'}],
 [{'Sample': 'O'}, {'sentence': 'O'}, {'2': 'O'}]]

In [ ]:
# Run all above
#df
'''
type(model_outputs)
for o0 in model_outputs:
    for o1 in o0:
        for o2 in o1:
            for o3 in o2:
                print(float(o3[0]))
'''
with open ("outputs/"+transformersmodel+"_"+keeper.ts+".pk", "wb") as wf:
    pk.dump(model_outputs, wf)
df = keeper.get_df()
keeper.keep(result)

results_2100.json


,eval_loss,precision,recall,f1_score,train,dev_test,training_epochs,transformer_model,timestamp
0,0.022889,0.886748,0.899882,0.893267,norne/data/no_bokmaal-ud-train_4c.bmes,norne/data/no_bokmaal-ud-dev_4c.bmes,1,bert-base-multilingual-cased,210817
1,0.022889,0.886748,0.899882,0.893267,norne/data/no_bokmaal-ud-train_4c.bmes,norne/data/no_bokmaal-ud-dev_4c.bmes,1,bert-base-multilingual-cased,210817


In [11]:
df

,eval_loss,precision,recall,f1_score,train,dev_test,training_epochs,transformer_model,timestamp
0,0.025114,0.870273,0.886256,0.878192,norne/data/no_bokmaal-ud-train_4c.bmes,norne/data/no_bokmaal-ud-dev_4c.bmes,1,KB/bert-base-swedish-cased,205613
1,0.022598,0.886946,0.901659,0.894242,norne/data/no_bokmaal-ud-train_4c.bmes,norne/data/no_bokmaal-ud-dev_4c.bmes,1,KB/bert-base-swedish-cased,210558
2,0.043999,0.898704,0.904028,0.901359,norne/data/no_bokmaal-ud-train_4c.bmes,norne/data/no_bokmaal-ud-dev_4c.bmes,8,bert-base-multilingual-cased,221223
3,0.030510,0.928614,0.932464,0.930535,norne/data/no_bokmaal-ud-train_4c.bmes,norne/data/no_bokmaal-ud-dev_4c.bmes,8,xlm-roberta-base,232231
4,0.039373,0.900059,0.906991,0.903511,norne/data/no_bokmaal-ud-train_4c.bmes,norne/data/no_bokmaal-ud-dev_4c.bmes,8,KB/bert-base-swedish-cased,001724
